In [59]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import linalg

In [60]:
L = 0.01 #length in [m]
nx, ny = 21,21
dx = L / (nx - 1)
dy = L / (ny - 1)

x = np.linspace(0.0, L, num=nx)
y = np.linspace(0.0, L, num=ny)

alpha = 1e-4

#Initial conditions
T0 = np.full((ny,nx), 20.0)
T0[0,:] = 100.0 #bottom boundary
T0[:,0] = 100.0 #left boundary

In [76]:
def btcs(T0, nt, dt, delta_space, alpha, T_boundary): 
    #where delta_space = dx = dy
    
    ny, nx = T0.shape
    M,N = nx - 2, ny - 2
    
    sigma = alpha*dt/delta_space**2
    
    #Create LHS Operator
    A = lhs_operator(M, N, sigma)
    #Convert points to 1d array
    T = map_2d_to_1d(T0)
    #points at center
    I, J = M//2, N//2
    
    #Integrate Over Time
    for n in range(nt):
        b = rhs_vector(T, M, N, sigma, T_boundary)
        T = linalg.solve(A, b)
        
        if T[J*M + I] >= 70.0:
            break
    print("Time-Step: ", n+1, "Temp at Center:", T[J*M + I], 
          "Time: ", dt*(n+1), "[sec]")
    return map_1d_to_2d(T, nx, ny, T_boundary)

In [72]:
def lhs_operator(M, N, sigma):
    A = np.zeros((M * N, M * N))
    for j in range(N):
        for i in range(M):
            I = j * M + i
            west = I - 1
            east = I + 1
            north = I + M
            south = I - M
            
            if i==0 and j==0: #bottom-left corner (interior)
                A[I, I] = 1 / sigma + 4
                A[I, east] = -1.0
                A[I, north] = -1.0
            elif i == M - 1 and j==0: #bottom-right corner
                A[I, I] = 1 / sigma + 3
                A[I, west] = -1.0
                A[I, north] = -1.0
            elif i == 0 and j == N - 1: #top-left corner (interior)
                A[I, I] = 1 / sigma + 3
                A[I, east]  = -1.0
                A[I, south] = -1.0
            elif i == M - 1 and j == N - 1: #top-right corner
                A[I,I] = 1 / sigma + 2
                A[I, west] = -1.0
                A[I, south] = -1.0
            elif j == 0: #first row of interior points
                A[I, I] = 1 / sigma + 4
                A[I, west] = -1.0
                A[I, east] = -1.0
                A[I, north] = -1.0
            elif j == N - 1: #last row of interior points
                A[I, I] = 1 / sigma + 3
                A[I, south] = -1.0
                A[I, west] = -1.0
                A[I, east] = -1.0
            elif i == 0: #first column of interior points
                A[I, I] = 1 / sigma + 4
                A[I, south] = -1.0
                A[I, east] = -1.0
                A[I, north] = -1.0
            elif i == M - 1: #last column of interior points
                A[I, I] = 1 / sigma + 3
                A[I, south] = -1.0
                A[I, west] = -1.0
                A[I, north] = -1.0
            else:
                A[I,I] = 1 / sigma + 4
                A[I, south] = -1.0
                A[I, west] = -1.0
                A[I, east] = -1.0
                A[I, north] = -1.0
    return A

In [63]:
def rhs_vector(T, M, N, sigma, T_boundary):
    b = 1.0 / sigma * T
    
    for j in range(N):
        for i in range(M):
            I = j*M + i
            if i == 0:
                b[I] += T_boundary
            if j == 0:
                b[I] += T_boundary
    return b

In [64]:
def map_2d_to_1d(T_2d):
    T = T_2d[1:-1, 1:-1].flatten()
    return T

In [78]:
def map_1d_to_2d(T_1d, nx, ny, T_boundary):
    T = np.zeros((nx, ny))
    T[1:-1,1:-1] = T_1d.reshape((ny-2, nx-2))
    #boundary conditions
    T[0,:] = T_boundary
    T[:,0] = T_boundary
    T[:,-1] = T[:,-2]
    T[-1,:] = T[-2,:]
    return T

In [79]:
L = 0.01 #length in [m]
nx, ny = 21,21
dx = L / (nx - 1)
dy = L / (ny - 1)

x = np.linspace(0.0, L, num=nx)
y = np.linspace(0.0, L, num=ny)

sigma = 0.25

nt = 500
dt = sigma * delta_space**2 / alpha

alpha = 1e-4
delta_space = dx #also equals dy because Lx=Ly and nx=ny

#Initial conditions
Tb = 100.0 #boundary temp
T0 = np.full((ny,nx), 20.0)
T0[0,:] = Tb #bottom boundary
T0[:,0] = Tb #left boundary

T_1 = btcs(T0, nt, dt, delta_space, alpha, Tb)

Time-Step:  257 Temp at Center: 70.00387455599684 Time:  0.16062499999999996 [sec]


In [80]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 16

In [ ]:
plt.figure(figsize=(8.0, 5.0))
plt.xlabel("")